In [13]:
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import os

In [7]:
train_df = pd.read_csv(os.path.join('data', "train.csv"))
train_df

In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

train_df = train_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin']).dropna()
train_df["Sex"] = LabelEncoder().fit_transform(train_df["Sex"])
train_df["Embarked"] = LabelEncoder().fit_transform(train_df["Embarked"])
train_df['Age'] = StandardScaler().fit_transform(train_df['Age'].values.reshape(-1, 1))
train_df['bias'] = np.ones(train_df.shape[0])

In [9]:
train_df

In [10]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['Survived'])
y = train_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.15, random_state=69)

In [15]:
y_train = y_train.reshape(-1, 1) 
y_test = y_test.reshape(-1, 1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [14]:


def MSELoss(y_true: np.ndarray, y_pred: np.ndarray) -> np.float32:
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return (1/len(y_true)) * np.sum((y_pred - y_true) ** 2)

def MAELoss(y_true: np.ndarray, y_pred: np.ndarray) -> np.float32:
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return (1/len(y_true)) * np.sum(np.abs((y_pred - y_true)))

def accuracy_with_thresholding(y_true: np.ndarray, y_pred: np.ndarray, threshold = 0.5):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return np.sum(
        y_true == (y_pred > threshold).astype(np.int32)
    ) / len(y_true)
    
def sigmoid(z):
    return 1/(1 + np.exp(-z))


In [15]:
import torch
import numpy as np

In [18]:
X = np.array([1])
Y = np.array([0])
Z = X + Y
X ,Y , Z 

In [19]:
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([0.0], requires_grad=True)
z = (x + y) **2
x, y, z

In [20]:
z.backward()

In [21]:
x.grad

In [22]:
y.grad

In [23]:
# Dataset

# Dataloader

# Model

# Traing-Valid Loop

# Test


In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TitanicDataset(Dataset):
    def __init__(self, df: pd.DataFrame, target: str, transforms=None):
        super().__init__()
        self.df = df
        self.features = df.drop(columns=[target]).values
        self.target = df[target].values.reshape(-1, 1)
        self.transforms = transforms
        
    def __len__(self):
        return len(self.target)
        
    def __getitem__(self, idx):
        X = torch.from_numpy(self.features[idx,:]).float()
        y = torch.from_numpy(self.target[idx,:]).float()
        
        if self.transforms:
            X = self.transforms(X)
        
        return (X, y)


In [41]:
training_df = pd.DataFrame(X_train, columns=train_df.columns - ["Survived"])
training_df['Survived'] = y_train
training_df

train_dataset = TitanicDataset(training_df, target='Survived')

In [42]:
val_df = pd.DataFrame(X_test, columns=train_df.columns - ["Survived"])
val_df['Survived'] = y_test
val_df

val_dataset = TitanicDataset(val_df, target='Survived')

In [43]:
len(train_dataset), len(val_dataset)

In [44]:
train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [89]:
class TitanicModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(7, 3)
        self.linear2 = torch.nn.Linear(3, 1)
        
    def forward(self, x):
        x = torch.relu(self.linear1(x))
        return torch.sigmoid(self.linear2(x))

In [124]:
# Training Loop
# Loss, Optimizer, epochs, learning rate, seed

torch.manual_seed(420)
model = TitanicModel()
learning_rate = 0.001
epochs = 1000
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [125]:
# model.to('cuda')
max_accuracy = 0
for i in range(epochs):
    for idx, (X_batch, y_batch) in enumerate(train_data_loader):
        # X_batch, y_batch = X_batch.to('cuda'), y_batch.to('cuda')
        model.train()
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        # y_pred = y_pred.to('cpu')
        # y_batch = y_batch.to('cpu')
        loss_value = loss(y_pred, y_batch)
        loss_value.backward()
        optimizer.step()
        
    # Validate
    if i % 100 == 0:
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_accuracy = 0
            for X_val, y_val in val_data_loader:
                y_pred = model(X_val)
                val_loss += loss(y_pred, y_val)
                val_accuracy += accuracy_with_thresholding(y_val.numpy(), y_pred.numpy())
            
            val_accuracy /= len(val_data_loader)
            val_loss /= len(val_data_loader)
            
            if val_accuracy > max_accuracy:
                max_accuracy = val_accuracy
            
            print(f"Epoch {i} - Training Loss: {loss_value.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy}")

In [126]:
max_accuracy

# Mnist

In [9]:
print('Penis')


In [29]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [31]:
transform = ToTensor()
mnist_ds = MNIST(root = 'data' ,download=True, transform=transform)
train_ds, val_ds = torch.utils.data.random_split(mnist_ds, [0.85, 0.15])

image, label = train_ds[0]


In [33]:
image

In [34]:
train_data_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_data_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

# Mnist

In [57]:
#Model
import torch.optim.sgd


class MnistModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(28*28, 512)
        self.linear2 = torch.nn.Linear(512, 128)
        self.linear3 = torch.nn.Linear(128, 32)
        self.linear4 = torch.nn.Linear(32, 10)
    
    def forward(self, a):
        a = torch.relu(self.linear1(a))
        a = torch.relu(self.linear2(a))
        a = torch.relu(self.linear3(a))
        a = torch.sigmoid(self.linear4(a))       
        return torch.softmax(a, 1)

# Training Loop
# Loss, Optimizer, epochs, learning rate, seed

torch.manual_seed(420)
model = MnistModel()
learning_rate = 0.01
epochs = 100
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [60]:
from torch.nn.functional import one_hot

model.train()
optimizer.zero_grad()
x_batch, y_batch = mnist_ds[0]
y_pred = model(x_batch.reshape(1, 784))
loss_value = loss(y_pred, one_hot(torch.tensor([y_batch]), 10).float())
loss_value.backward()
optimizer.step()

In [63]:
y_pred

tensor([[0.0853, 0.0915, 0.0823, 0.0905, 0.0999, 0.1318, 0.1183, 0.1004, 0.1005,
         0.0994]], grad_fn=<SoftmaxBackward0>)

In [64]:
y_pred.shape

torch.Size([1, 10])